# 🧠 量子ガイスターAI開発チュートリアル

このノートブックでは、PennyLaneとPyTorchを使った量子機械学習によるガイスターAIの開発方法を学びます。

## 📚 目次
1. [環境セットアップ](#1-環境セットアップ)
2. [基本的な量子回路](#2-基本的な量子回路)
3. [ガイスターゲームエンジン](#3-ガイスターゲームエンジン)
4. [量子AIアーキテクチャ](#4-量子aiアーキテクチャ)
5. [学習プロセス](#5-学習プロセス)
6. [性能評価と調整](#6-性能評価と調整)
7. [高度なカスタマイズ](#7-高度なカスタマイズ)

## 1. 環境セットアップ

必要なライブラリをインポートします。

In [ ]:
import sys
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pennylane as qml
import matplotlib.pyplot as plt
import random
from collections import deque

# プロジェクトパスを追加
sys.path.append(str(Path.cwd()))
sys.path.append(str(Path.cwd() / "src"))

print("📦 ライブラリ読み込み完了")
print(f"🔥 PyTorch: {torch.__version__}")
print(f"⚛️ PennyLane: {qml.__version__}")

## 2. 基本的な量子回路

量子ガイスターAIの基盤となる量子回路を理解しましょう。

In [ ]:
# 6量子ビット量子デバイスを作成
n_qubits = 6
dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev)
def simple_quantum_circuit(inputs):
    """
    シンプルな量子回路の例
    
    Args:
        inputs: 入力パラメータ（6次元）
    
    Returns:
        量子状態の期待値
    """
    # 入力エンコーディング
    for i in range(n_qubits):
        qml.RY(inputs[i], wires=i)
    
    # エンタングリング層
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    
    # 測定
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# テスト実行
test_input = np.random.rand(6) * np.pi
result = simple_quantum_circuit(test_input)
print(f"入力: {test_input[:3]:.3f}...")
print(f"出力: {result[:3]:.3f}...")

## 3. ガイスターゲームエンジン

ガイスターゲームの基本的な操作を理解しましょう。

In [ ]:
from qugeister import GeisterEngine

# ゲームエンジンを初期化
game = GeisterEngine()
game.reset_game()

print("🎮 ガイスターゲーム初期化完了")
print(f"現在のプレイヤー: {game.current_player}")
print(f"ゲーム状態: {'終了' if game.game_over else '進行中'}")

# 合法手を取得
legal_moves = game.get_legal_moves('A')
print(f"プレイヤーAの合法手数: {len(legal_moves)}")
print(f"最初の3手: {legal_moves[:3]}")

# ボード状態の可視化
def visualize_board(game):
    """
    ゲームボードを簡単に可視化
    """
    board = game.get_board_state()
    print("\n📋 ボード状態 (6x6):")
    for row in board:
        print(' '.join([f'{cell:2.0f}' for cell in row]))

visualize_board(game)

## 4. 量子AIアーキテクチャ

CNN風設計の量子AIアーキテクチャを構築します。

In [ ]:
# 既存の量子AIクラスをインポート
from test_qubits_6 import QuantumBattleAI_6Qubits

class CustomQuantumAI(nn.Module):
    """
    カスタマイズ可能な量子AI
    
    このクラスでは学習の各部分を細かく制御できます。
    """
    
    def __init__(self, n_qubits=6, n_layers=2):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        
        # 量子デバイス
        self.dev = qml.device('default.qubit', wires=n_qubits)
        
        # 前処理層（CNN風）
        self.conv1 = nn.Conv2d(7, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # 量子回路パラメータ
        self.quantum_params = nn.Parameter(
            torch.randn(n_layers, n_qubits) * 0.1
        )
        
        # 後処理層
        self.fc1 = nn.Linear(n_qubits, 64)
        self.fc2 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)  # Q値出力
        
    def quantum_circuit(self, inputs, params):
        """
        カスタマイズ可能な量子回路
        """
        # 入力エンコーディング
        for i in range(self.n_qubits):
            qml.RY(inputs[i], wires=i)
        
        # パラメータ化された層
        for layer in range(self.n_layers):
            # 回転ゲート
            for i in range(self.n_qubits):
                qml.RY(params[layer, i], wires=i)
            
            # エンタングリング
            for i in range(self.n_qubits - 1):
                qml.CNOT(wires=[i, i + 1])
            qml.CNOT(wires=[self.n_qubits - 1, 0])  # 循環結合
        
        return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]
    
    def forward(self, game_state):
        """
        フォワードパス
        """
        batch_size = game_state.size(0)
        
        # CNN処理
        x = torch.relu(self.conv1(game_state))
        x = torch.relu(self.conv2(x))
        x = self.global_pool(x).view(batch_size, -1)
        
        # 量子処理
        quantum_outputs = []
        for i in range(batch_size):
            quantum_input = torch.tanh(x[i]) * np.pi / 2
            
            # 量子回路をQNodeとして作成
            @qml.qnode(self.dev)
            def circuit(inputs, params):
                return self.quantum_circuit(inputs, params)
            
            q_out = circuit(quantum_input, self.quantum_params)
            quantum_outputs.append(torch.stack(q_out))
        
        quantum_tensor = torch.stack(quantum_outputs)
        
        # 後処理
        x = torch.relu(self.fc1(quantum_tensor))
        x = torch.relu(self.fc2(x))
        q_value = self.output(x)
        
        return q_value

# カスタムAIをテスト
custom_ai = CustomQuantumAI(n_qubits=6, n_layers=3)
print(f"🤖 カスタム量子AI作成完了")
print(f"パラメータ数: {sum(p.numel() for p in custom_ai.parameters())}")

## 5. 学習プロセス

量子AIの学習プロセスをステップバイステップで理解しましょう。

In [ ]:
def custom_training_loop(model, episodes=100, batch_size=8):
    """
    カスタマイズ可能な学習ループ
    
    この関数では学習の各ステップを詳細に制御できます。
    """
    # 学習設定
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()
    
    # 経験リプレイバッファー
    replay_buffer = deque(maxlen=1000)
    
    # 学習統計
    episode_rewards = []
    losses = []
    
    print(f"🎯 学習開始: {episodes}エピソード")
    
    for episode in range(episodes):
        # ゲーム初期化
        game = GeisterEngine()
        game.reset_game()
        
        episode_reward = 0
        step_count = 0
        max_steps = 50  # エピソード長制限
        
        while not game.game_over and step_count < max_steps:
            current_player = game.current_player
            legal_moves = game.get_legal_moves(current_player)
            
            if not legal_moves:
                break
            
            if current_player == 'A':  # AIの手番
                # 状態をエンコード
                state = game.get_board_state()
                state_tensor = torch.FloatTensor(state).unsqueeze(0)  # チャンネル次元追加
                
                # 7チャンネル形式に変換（簡略化）
                state_7ch = torch.zeros(1, 7, 6, 6)
                state_7ch[0, 0] = state_tensor[0]  # 基本ボード情報
                
                # ε-greedy行動選択
                epsilon = max(0.01, 0.5 * (0.995 ** episode))
                
                if random.random() < epsilon:
                    # ランダム行動
                    action = random.choice(legal_moves)
                else:
                    # AIによる行動選択（簡略化）
                    action = random.choice(legal_moves)
                
                # 行動実行
                game.make_move(current_player, action)
                
                # 報酬計算
                if game.game_over:
                    if game.winner == 'A':
                        reward = 100  # 勝利
                    elif game.winner == 'B':
                        reward = -100  # 敗北
                    else:
                        reward = 0  # 引き分け
                else:
                    reward = 1  # 継続報酬
                
                episode_reward += reward
                
                # 経験をバッファーに保存
                replay_buffer.append((state_7ch, action, reward))
                
            else:  # 相手（ランダム）の手番
                action = random.choice(legal_moves)
                game.make_move(current_player, action)
            
            step_count += 1
        
        episode_rewards.append(episode_reward)
        
        # バッチ学習
        if len(replay_buffer) >= batch_size and episode % 10 == 0:
            # ランダムサンプリング
            batch = random.sample(replay_buffer, batch_size)
            
            # バッチデータを準備
            states = torch.cat([item[0] for item in batch])
            rewards = torch.FloatTensor([item[2] for item in batch])
            
            # フォワードパス
            optimizer.zero_grad()
            q_values = model(states).squeeze()
            
            # 損失計算
            loss = loss_fn(q_values, rewards)
            losses.append(loss.item())
            
            # バックプロパゲーション
            loss.backward()
            optimizer.step()
        
        # 進捗表示
        if episode % 20 == 0:
            avg_reward = np.mean(episode_rewards[-20:]) if episode_rewards else 0
            avg_loss = np.mean(losses[-10:]) if losses else 0
            print(f"Episode {episode}: 平均報酬={avg_reward:.1f}, 損失={avg_loss:.4f}")
    
    return episode_rewards, losses

# 短期学習テスト
print("🧪 短期学習テスト開始...")
rewards, losses = custom_training_loop(custom_ai, episodes=50, batch_size=4)

# 結果可視化
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(rewards)
plt.title('エピソード報酬')
plt.xlabel('エピソード')
plt.ylabel('報酬')

plt.subplot(1, 2, 2)
if losses:
    plt.plot(losses)
    plt.title('学習損失')
    plt.xlabel('更新ステップ')
    plt.ylabel('損失')

plt.tight_layout()
plt.show()

print(f"✅ 学習完了: 平均報酬 = {np.mean(rewards):.2f}")

## 6. 性能評価と調整

学習済みモデルの性能を評価し、ハイパーパラメータを調整します。

In [ ]:
def evaluate_model(model, n_games=10):
    """
    モデルの性能を評価
    """
    model.eval()
    wins = 0
    draws = 0
    losses = 0
    
    with torch.no_grad():
        for game_idx in range(n_games):
            game = GeisterEngine()
            game.reset_game()
            
            step_count = 0
            max_steps = 50
            
            while not game.game_over and step_count < max_steps:
                current_player = game.current_player
                legal_moves = game.get_legal_moves(current_player)
                
                if not legal_moves:
                    break
                
                # 両プレイヤーともランダム（デモ用）
                action = random.choice(legal_moves)
                game.make_move(current_player, action)
                step_count += 1
            
            # 結果集計
            if game.winner == 'A':
                wins += 1
            elif game.winner == 'B':
                losses += 1
            else:
                draws += 1
    
    model.train()
    
    return {
        'wins': wins,
        'draws': draws, 
        'losses': losses,
        'win_rate': wins / n_games * 100
    }

# 性能評価
results = evaluate_model(custom_ai, n_games=20)
print("📊 性能評価結果:")
print(f"勝利: {results['wins']}/20 ({results['win_rate']:.1f}%)")
print(f"引分: {results['draws']}/20")
print(f"敗北: {results['losses']}/20")

# ハイパーパラメータ調整のヒント
print("\n🔧 ハイパーパラメータ調整のヒント:")
print("1. 学習率: 0.001-0.01 (高すぎると不安定、低すぎると遅い)")
print("2. バッチサイズ: 8-32 (メモリとの兼ね合い)")
print("3. エピソード数: 1000-5000 (十分な学習のため)")
print("4. ε値: 0.1-0.5 開始、0.01まで減衰 (探索と活用のバランス)")
print("5. 量子回路層数: 2-5層 (表現力と計算コストのトレードオフ)")

## 7. 高度なカスタマイズ

より高度な量子回路設計や学習手法を試してみましょう。

In [ ]:
# 高度な量子回路設計
class AdvancedQuantumCircuit:
    """
    より複雑な量子回路設計
    """
    
    def __init__(self, n_qubits=6):
        self.n_qubits = n_qubits
        self.dev = qml.device('default.qubit', wires=n_qubits)
    
    def amplitude_encoding_circuit(self, data, params):
        """
        振幅エンコーディングを使用した回路
        """
        # 正規化
        norm = torch.norm(data)
        if norm > 0:
            data = data / norm
        
        # 振幅エンコーディング（簡略版）
        for i in range(self.n_qubits):
            qml.RY(data[i] * np.pi, wires=i)
        
        # Strongly Entangling Layers
        qml.StronglyEntanglingLayers(params, wires=range(self.n_qubits))
        
        return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]
    
    def variational_circuit(self, inputs, params):
        """
        変分量子回路
        """
        # データエンコーディング
        for i in range(self.n_qubits):
            qml.Hadamard(wires=i)
            qml.RZ(inputs[i], wires=i)
        
        # 変分層
        for layer in range(len(params)):
            # 回転ゲート
            for i in range(self.n_qubits):
                qml.RX(params[layer][i][0], wires=i)
                qml.RY(params[layer][i][1], wires=i)
                qml.RZ(params[layer][i][2], wires=i)
            
            # エンタングリング
            for i in range(self.n_qubits - 1):
                qml.CNOT(wires=[i, i + 1])
            if self.n_qubits > 2:
                qml.CNOT(wires=[self.n_qubits - 1, 0])
        
        return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]

# 高度な学習手法
def parameter_shift_training(model, data, target, lr=0.01):
    """
    パラメータシフト法による勾配計算
    
    量子回路特有の勾配計算手法
    """
    gradients = []
    
    for param in model.parameters():
        if param.requires_grad:
            param_grad = torch.zeros_like(param)
            
            for i in range(param.numel()):
                # パラメータを+π/2シフト
                param.data.view(-1)[i] += np.pi / 2
                output_plus = model(data)
                
                # パラメータを-π/2シフト
                param.data.view(-1)[i] -= np.pi
                output_minus = model(data)
                
                # 元に戻す
                param.data.view(-1)[i] += np.pi / 2
                
                # 勾配計算
                grad = (output_plus - output_minus) / 2
                param_grad.view(-1)[i] = grad.item()
            
            gradients.append(param_grad)
    
    return gradients

# 量子回路の可視化
def visualize_quantum_circuit():
    """
    量子回路の構造を可視化
    """
    n_qubits = 4  # 可視化用に小さくする
    dev = qml.device('default.qubit', wires=n_qubits)
    
    @qml.qnode(dev)
    def demo_circuit(params):
        # データエンコーディング
        for i in range(n_qubits):
            qml.RY(params[i], wires=i)
        
        # エンタングリング
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        
        return qml.expval(qml.PauliZ(0))
    
    # 回路図を生成
    params = np.random.rand(n_qubits)
    print("🔧 量子回路の構造:")
    print(qml.draw(demo_circuit)(params))

visualize_quantum_circuit()

print("\n🎓 学習のポイント:")
print("1. 量子回路の深さ: 浅い回路から始めて徐々に複雑にする")
print("2. エンコーディング方法: Angle, Amplitude, IQPなど様々な手法を試す")
print("3. 測定方法: PauliZ, PauliX, PauliYの組み合わせで情報を抽出")
print("4. バリエーショナル層: 表現力と学習可能性のバランス")
print("5. ノイズ耐性: 実機では量子ノイズを考慮した設計が重要")

## 🎯 実践的な使い方

### 基本的な学習フロー:

1. **データ準備**: ゲーム状態を7チャンネル形式で表現
2. **回路設計**: 問題に応じて量子回路を設計
3. **学習実行**: DQNまたはカスタム学習ループで訓練
4. **評価**: 対戦テストで性能を確認
5. **調整**: ハイパーパラメータを最適化

### パラメータ調整の指針:

- **学習が不安定**: 学習率を下げる、バッチサイズを小さくする
- **学習が遅い**: 学習率を上げる、エピソード数を増やす
- **過学習**: 正則化を強める、回路を浅くする
- **汎化性能が低い**: データ拡張、ドロップアウト追加

### 次のステップ:

1. 実際の量子デバイス（IBM Quantum、Rigetti）での実行
2. より複雑なゲーム環境での評価
3. 他の量子機械学習アルゴリズムとの比較
4. ハイブリッド古典-量子アーキテクチャの探索